# **Part 1:  Document Conversion, OCR, and Preprocessing**

In [ ]:
!pip install pytesseract pdf2image pillow

In [ ]:
!apt-get update
!apt-get install -y tesseract-ocr
!apt-get install -y tesseract-ocr-deu
!apt-get install -y poppler-utils

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [53.5 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [917 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:9 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [2,439 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/multiverse amd64 Packages [51.5 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,389 kB]

In [ ]:
!which tesseract

/usr/bin/tesseract


Document Conversion from pdf to txt using OCR

In [ ]:
import pytesseract
from pdf2image import convert_from_path
import os

In [ ]:
pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'

In [ ]:
def pdf_to_text(pdf_path, output_txt_path, language='deu'):
    images = convert_from_path(pdf_path)

    full_text = ""

    for i, image in enumerate(images):
        text = pytesseract.image_to_string(image, lang=language)
        full_text += text

    with open(output_txt_path, 'w', encoding='utf-8') as f:
        f.write(full_text)

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
for i in range(1,5):
  pdf_path        = f"/content/drive/MyDrive/sample{i}.pdf"
  output_txt_path = f"/content/drive/MyDrive/output{i}.txt"
  pdf_to_text(pdf_path, output_txt_path)

Preprocessing

In [ ]:
import re
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize

# Load NLTK resources
nltk.download('punkt')

def preprocess_text(file_path):
  with open(file_path, 'r', encoding='utf-8') as file:
    text = file.read()

  # Text cleaning
  text = re.sub(r'\n', ' ', text)  # Replace newline characters with space
  text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
  text = text.strip()  # Remove leading and trailing whitespace

  # Sentence segmentation
  sentences = sent_tokenize(text)

  # Tokenization
  tokenized_sentences = [word_tokenize(sentence) for sentence in sentences]

  return tokenized_sentences

preprocessed_text = []
# Example usage
for i in range(1,5):

  file_path = f'/content/drive/MyDrive/output{i}.txt'
  preprocessed_text.append(preprocess_text(file_path))

[print(preprocessed_text[i]) for i in range(4)]


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


[['[', 'Ihr', 'Logo', ']', 'Musterfirma', '—-', 'Musterstraße', '23', '—', '12345', ',', 'Musterhausen', 'Gutschriftsempfänger', 'Straße', ',', 'Hausnummer', 'PLZ', ',', 'Ort', 'Musterfirma', 'AG', 'Musterstraße', ',', '23', '12345', ',', 'Musterhausen', 'Datum', ':', '01.03.2019', 'Gutschrift', 'Nr', '.'], [':', '2019-1004', 'Ihre', 'UmSt', '.'], ['ID', ':', 'DE123456789', 'Gutschriftssdatum', 'entspricht', 'Liefer-/Leistungsdatum', 'Gutschrift', 'Sehr', 'geehrter', 'Herr', 'Schmidt', ',', 'gemäß', 'unserer', 'Vereinbarung', 'schreiben', 'wir', 'Ihnen', 'folgende', 'Leistungen', 'gut', ':', 'Position', 'Anzahl', 'Einheit', 'Bezeichnung', 'Einzelpreis', 'Gesamtpreis', '1', '5', 'Stück', 'Musterleistung', '3,00', '€', '15,00', '€', '2', '3', 'Stück', 'Musterleistung', '5,00', '€', '15,00', '€', 'Nettopreis', '30,00', '€', 'Zzgl', '.'], ['19', '%', 'USt', '.'], ['5,70€', 'Gutschriftbetrag', '35,70', '€', 'Wir', 'überweisen', 'Ihnen', 'den', 'Gutschriftbetrag', 'in', 'den', 'nächsten', 'T

[None, None, None, None]

# **Part 2: LLM-Powered Understanding and Actions**

In [ ]:
!pip install openai==0.28

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 3.1 MB/s eta 0:00:00


In [ ]:
import openai

openai.api_key = "API_KEY"

text = preprocessed_text

In [ ]:
def extract_relationships(text):
    response = openai.Completion.create(
        engine="gpt-3.5-turbo-instruct",
        prompt=f"Extract relationships between entities from the following text make key value pair:\n\n{text}",
        max_tokens=500
    )
    relationships = response.choices[0].text.strip()
    return relationships

relationships = []
for i in range(3):
  relationships.append(extract_relationships(text[i]))

In [ ]:
relationships.append(extract_relationships(text[3]))

In [ ]:
[print(relationships[i]+'\n') for i in range(4)]

- Company name: Musterfirma
- Address: Musterstraße 23, 12345 Musterhausen
- Invoice number: 2019-1004
- VAT number: DE123456789
- Invoice date: 01.03.2019
- Customer: Herr Schmidt
- Payment method: bank transfer
- Account information: Konto
- Amount: 35,70 €
- Delivery date: 01.03.2019
- Item 1: Musterleistung
- Quantity: 5
- Unit price: 3,00 €
- Total price: 15,00 €
- Item 2: Musterleistung
- Quantity: 3
- Unit price: 5,00 €
- Total price: 15,00 €
- Net price: 30,00 €
- VAT rate: 19%
- VAT amount: 5,70 €
- Bank: Commerzbank
- VAT ID: DE24324567
- IBAN: DE3423 4562 3435 6765
- Register court: Amtsgericht Charlottenburg
- Company email: info@muster.de
- Website: www.firma.de
- Owner/Managing Director: Max Mustermann
- Phone number: +40 (0) 30 12345678
- Title: Inhaber/Geschäftsführer
- Contact email: info@muster.de
- Contact website: www.firma.de

"Billing Information": {'Date': '06/10/2021', 'Invoice Number': 'LS353348', 'Customer Name': 'Donec odio', 'Address': '84746 Buhler St, 3207

[None, None, None, None]

In [ ]:
[print(preprocessed_text[i])for i in range(3)]

[['[', 'Ihr', 'Logo', ']', 'Musterfirma', '—-', 'Musterstraße', '23', '—', '12345', ',', 'Musterhausen', 'Gutschriftsempfänger', 'Straße', ',', 'Hausnummer', 'PLZ', ',', 'Ort', 'Musterfirma', 'AG', 'Musterstraße', ',', '23', '12345', ',', 'Musterhausen', 'Datum', ':', '01.03.2019', 'Gutschrift', 'Nr', '.'], [':', '2019-1004', 'Ihre', 'UmSt', '.'], ['ID', ':', 'DE123456789', 'Gutschriftssdatum', 'entspricht', 'Liefer-/Leistungsdatum', 'Gutschrift', 'Sehr', 'geehrter', 'Herr', 'Schmidt', ',', 'gemäß', 'unserer', 'Vereinbarung', 'schreiben', 'wir', 'Ihnen', 'folgende', 'Leistungen', 'gut', ':', 'Position', 'Anzahl', 'Einheit', 'Bezeichnung', 'Einzelpreis', 'Gesamtpreis', '1', '5', 'Stück', 'Musterleistung', '3,00', '€', '15,00', '€', '2', '3', 'Stück', 'Musterleistung', '5,00', '€', '15,00', '€', 'Nettopreis', '30,00', '€', 'Zzgl', '.'], ['19', '%', 'USt', '.'], ['5,70€', 'Gutschriftbetrag', '35,70', '€', 'Wir', 'überweisen', 'Ihnen', 'den', 'Gutschriftbetrag', 'in', 'den', 'nächsten', 'T

[None, None, None]

In [ ]:
prev_examples = []
for i in range(4):
  prev_examples.append('prompt\n-----------\n'+str(preprocessed_text[i])+'\n-----------\n\n'+'response\n-----------\n'+relationships[i]+'\n-----------')

In [ ]:
print(prev_examples)

["prompt\n-----------\n[['[', 'Ihr', 'Logo', ']', 'Musterfirma', '—-', 'Musterstraße', '23', '—', '12345', ',', 'Musterhausen', 'Gutschriftsempfänger', 'Straße', ',', 'Hausnummer', 'PLZ', ',', 'Ort', 'Musterfirma', 'AG', 'Musterstraße', ',', '23', '12345', ',', 'Musterhausen', 'Datum', ':', '01.03.2019', 'Gutschrift', 'Nr', '.'], [':', '2019-1004', 'Ihre', 'UmSt', '.'], ['ID', ':', 'DE123456789', 'Gutschriftssdatum', 'entspricht', 'Liefer-/Leistungsdatum', 'Gutschrift', 'Sehr', 'geehrter', 'Herr', 'Schmidt', ',', 'gemäß', 'unserer', 'Vereinbarung', 'schreiben', 'wir', 'Ihnen', 'folgende', 'Leistungen', 'gut', ':', 'Position', 'Anzahl', 'Einheit', 'Bezeichnung', 'Einzelpreis', 'Gesamtpreis', '1', '5', 'Stück', 'Musterleistung', '3,00', '€', '15,00', '€', '2', '3', 'Stück', 'Musterleistung', '5,00', '€', '15,00', '€', 'Nettopreis', '30,00', '€', 'Zzgl', '.'], ['19', '%', 'USt', '.'], ['5,70€', 'Gutschriftbetrag', '35,70', '€', 'Wir', 'überweisen', 'Ihnen', 'den', 'Gutschriftbetrag', 'in'

In [ ]:
prompt = "A model that takes in text details of document in german(this can be recipt), and responds with a english classification of document(example recipet or certificate) it also translates it and explains in english the response should be in english."
temperature = .4

In [ ]:
def generate_system_message(prompt):

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-16k",
        messages=[
          {
            "role": "system",
            "content": "You will be given a high-level description of the model we are training, and from that, you will generate a simple system prompt for that model to use. Remember, you are not generating the system message for data generation -- you are generating the system message to use for inference. A good format to follow is `Given $INPUT_DATA, you will $WHAT_THE_MODEL_SHOULD_DO.`.\n\nMake it as concise as possible. Include nothing but the system prompt in your response.\n\nFor example, never write: `\"$SYSTEM_PROMPT_HERE\"`.\n\nIt should be like: `$SYSTEM_PROMPT_HERE`."
          },
          {
              "role": "user",
              "content": prompt.strip(),
          }
        ],
        temperature=temperature,
        max_tokens=500,
    )

    return response.choices[0].message['content']

system_message = generate_system_message(prompt)

print(f'The system message is: `{system_message}`. Feel free to re-run this cell if you want a better result.')

The system message is: `Given a text document in German, you will classify the document (e.g., receipt or certificate) in English, translate it, and provide an English explanation.`. Feel free to re-run this cell if you want a better result.


In [ ]:
import pandas as pd

# Initialize lists to store prompts and responses
prompts = []
responses = []

# Parse out prompts and responses from examples
for example in prev_examples:
  try:
    split_example = example.split('-----------')
    prompts.append(split_example[1].strip())
    responses.append(split_example[3].strip())
  except:
    pass

# Create a DataFrame
df = pd.DataFrame({
    'prompt': prompts,
    'response': responses
})

# Remove duplicates
df = df.drop_duplicates()

print('There are ' + str(len(df)) + ' successfully-generated examples. Here are the first few:')

df.head()

There are 4 successfully-generated examples. Here are the first few:


,prompt,response
0,"[['[', 'Ihr', 'Logo', ']', 'Musterfirma', '—-'...",- Company name: Musterfirma\n- Address: Muster...
1,"[['06/10/2021', '.'], ['LS353348', ')', 'kmey'...","""Billing Information"": {'Date': '06/10/2021', ..."
2,"[['Firmenlogo', 'Max', 'Mustermann', '-', 'Mus...","{\n 'Name': 'Max Mustermann',\n 'Address..."
3,"[['Kraxi', 'GmbH', 'Flugzeugallee', '17', '123...",1. Entity 1: Kraxi GmbH\n Entity 2: Flugzeug...


In [ ]:
# Split the data into train and test sets, with 90% in the train set
train_df = df.sample(frac=0.9, random_state=42)
test_df = df.drop(train_df.index)

# Save the dataframes to .jsonl files
train_df.to_json('train.jsonl', orient='records', lines=True)
test_df.to_json('test.jsonl', orient='records', lines=True)

In [ ]:
system_message = "Given a text document in German, you will classify the document (e.g., receipt or certificate) in English, translate it, and provide an English explanation."

In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [ ]:
model_name = "NousResearch/llama-2-7b-chat-hf"  # or "meta-llama/Llama-2-7b-chat-hf" with Hugging Face key
dataset_name = "/content/drive/MyDrive/train.jsonl"
new_model = "llama-2-7b-custom"
lora_r = 64
lora_alpha = 16
lora_dropout = 0.1
use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False
output_dir = "./results"
num_train_epochs = 1
fp16 = False
bf16 = False
per_device_train_batch_size = 4
per_device_eval_batch_size = 4
gradient_accumulation_steps = 1
gradient_checkpointing = True
max_grad_norm = 0.3
learning_rate = 2e-4
weight_decay = 0.001
optim = "paged_adamw_32bit"
lr_scheduler_type = "constant"
max_steps = -1
warmup_ratio = 0.03
group_by_length = True
save_steps = 25
logging_steps = 5
max_seq_length = None
packing = False
device_map = {"":0}

In [ ]:
train_dataset = load_dataset('json', data_files={'train': dataset_name})
valid_dataset = load_dataset('json', data_files={'validation': dataset_name})

# Preprocess datasets
train_dataset_mapped = train_dataset.map(lambda examples: {'text': [f'[INST] <<SYS>>\n{system_message.strip()}\n<</SYS>>\n\n' + prompt + ' [/INST] ' + response for prompt, response in zip(examples['prompt'], examples['response'])]}, batched=True)
valid_dataset_mapped = valid_dataset.map(lambda examples: {'text': [f'[INST] <<SYS>>\n{system_message.strip()}\n<</SYS>>\n\n' + prompt + ' [/INST] ' + response for prompt, response in zip(examples['prompt'], examples['response'])]}, batched=True)

compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="all",
    evaluation_strategy="steps",
    eval_steps=5  # Evaluate every 5 steps
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset_mapped['train'],
    eval_dataset=valid_dataset_mapped['validation'],
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train the model
trainer.train()
trainer.model.save_pretrained(new_model)

# Test the model
logging.set_verbosity(logging.CRITICAL)
prompt = f"[INST] <<SYS>>\n{system_message}\n<</SYS>>\n\n ich bin harsh . [/INST]"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(prompt)
print(result[0]['generated_text'])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_ree

Step,Training Loss,Validation Loss


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


[INST] <<SYS>>
Given a text document in German, you will classify the document (e.g., receipt or certificate) in English, translate it, and provide an English explanation.
<</SYS>>

 ich bin harsh . [/INST]  Based on the text you provided, I can classify the document as a receipt. Here is the English translation:

"I am harsh"

Explanation: The text "ich bin harsh" is a German sentence that translates to "I am harsh" in English. It is a simple statement expressing a personal quality or trait.


In [ ]:
from transformers import pipeline

prompt = f"[INST] <<SYS>>\n{system_message}\n<</SYS>>\n\n[['[', 'Ihr', 'Logo', ']', 'Musterfirma', '—-', 'Musterstraße', '23', '—', '12345', ',', 'Musterhausen', 'Gutschriftsempfänger', 'Straße', ',', 'Hausnummer', 'PLZ', ',', 'Ort', 'Musterfirma', 'AG', 'Musterstraße', ',', '23', '12345', ',', 'Musterhausen', 'Datum', ':', '01.03.2019', 'Gutschrift', 'Nr', '.'], [':', '2019-1004', 'Ihre', 'UmSt', '.'], ['ID', ':', 'DE123456789', 'Gutschriftssdatum', 'entspricht', 'Liefer-/Leistungsdatum', 'Gutschrift', 'Sehr', 'geehrter', 'Herr', 'Schmidt', ',', 'gemäß', 'unserer', 'Vereinbarung', 'schreiben', 'wir', 'Ihnen', 'folgende', 'Leistungen', 'gut', ':', 'Position', 'Anzahl', 'Einheit', 'Bezeichnung', 'Einzelpreis', 'Gesamtpreis', '1', '5', 'Stück', 'Musterleistung', '3,00', '€', '15,00', '€', '2', '3', 'Stück', 'Musterleistung', '5,00', '€', '15,00', '€', 'Nettopreis', '30,00', '€', 'Zzgl', '.'], ['19', '%', 'USt', '.'], ['5,70€', 'Gutschriftbetrag', '35,70', '€', 'Wir', 'überweisen', 'Ihnen', 'den', 'Gutschriftbetrag', 'in', 'den', 'nächsten', 'Tagen', 'auf', 'Ihr', 'Konto', '.'], ['Mit', 'freundlichen', 'Grüßen', 'Max', 'Mustermann', 'Musterfirma', 'GmbH', 'Kreditinstitut', ':', 'Commerzbank', 'USt-ID', ':', 'DE24324567', 'Musterstraße', ',', '23', 'IBAN', ':', 'DE3423', '4562', '3435', '6765', 'HRB', ':', '1234567B', '12345', ',', 'Musterhausen', 'BIC', ':', 'COBADEFFXXX', 'Amtsgericht', ':', 'Charlottenburg', 'Tel', ':', '+40', '(', '0', ')', '30', '12345678', 'Kto', '.'], ['Inh', '.'], [':', 'Max', 'Mustermann', 'Geschäftsführer', ':', 'Max', 'Mustermann', 'E-Mail', ':', 'info', '@', 'muster.de', 'Webseite', ':', 'www.firma.de']]. [/INST]" # replace the command here with something relevant to your task
num_new_tokens = 1000  # change to the number of new tokens you want to generate

# Count the number of tokens in the prompt
num_prompt_tokens = len(tokenizer(prompt)['input_ids'])

# Calculate the maximum length for the generation
max_length = num_prompt_tokens + num_new_tokens

gen = pipeline('text-generation', model=model, tokenizer=tokenizer, max_length=max_length)
result = gen(prompt)
print(result[0]['generated_text'].replace(prompt, ''))

  Based on the provided text document in German, I can classify it as a receipt or certificate. The document appears to be a Gutschrift, which is a type of invoice or receipt in Germany.

The document includes various details such as the recipient's name and address, the date, and the amount of the Gutschrift. It also includes information about the goods or services provided, the prices, and the total amount due.

Here is a rough translation of the document into English:

"Dear Sir/Madam,

We are pleased to inform you that we have issued a Gutschrift for the following goods/services:

* Position: 3 Stück
* Anzahl: 5 Stück
* Einheit: Musterleistung
* Bezeichnung: Musterleistung
* Einzelpreis: 3,00 €
* Gesamtpreis: 15,00 €
* Nettopreis: 30,00 €

The Gutschrift amount is 5,70 €. We will transfer the Gutschrift amount to your account in the next few days.

Please find attached the Gutschrift for your reference.

Sincerely,
Max Mustermann
Musterfirma GmbH & Co. KG
Commerzbank USt-ID: DE2432

In [ ]:
# Function to free GPU memory
def free_memory():
    import gc
    gc.collect()
    torch.cuda.empty_cache()
free_memory()

In [ ]:
# Merge and save the fine-tuned model

model_path = "/content/llama-2-7b-custom"  # change to your preferred path

# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="cpu",
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Save the merged model
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

('/content/llama-2-7b-custom/tokenizer_config.json',
 '/content/llama-2-7b-custom/special_tokens_map.json',
 '/content/llama-2-7b-custom/tokenizer.model',
 '/content/llama-2-7b-custom/added_tokens.json',
 '/content/llama-2-7b-custom/tokenizer.json')